<a href="https://colab.research.google.com/github/yardsale8/probability_simulations_in_R/blob/main/2_2_simulating_the_geometric_and_negative_binomial_distributions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
library(dplyr)
library(tidyr)
library(purrr)
library(devtools)
install_github('yardsale8/purrrfect', force = TRUE)
library(purrrfect)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: usethis





── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/Rtmp8Q4bPn/remotesfd63e50761/yardsale8-purrrfect-d91fae7/DESCRIPTION’ ... OK
* preparing ‘purrrfect’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* building ‘purrrfect_1.0.1.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


Attaching package: ‘purrrfect’


The following objects are masked from ‘package:base’:

    replicate, tabulate




# Simulating the Geometric and Negative Binomial Distributions

In this notebook, we will explore two more distributions related to a Bernoulli process, namely the geometric and negative binomial distributions.  Unlike the binomial distribution--which counts a variable number of success over a fixed numer of trials--these distributions continue generating Bernoulli trials until reaching a fixed number of successes.  We call the associate questions **waiting time problems**.

## Review - Bernoulli Trials and Waiting Time Distributions

Before we proceed simulating waiting time problems for a Bernoulli process, let's review this processes and the related distributions.

### Review - Bernoulli Process

A simple outcomes is generated by a [Bernoulli process](https://en.wikipedia.org/wiki/Bernoulli_process), provided
1. Outcomes are independent,
2. There are two possible outcomes (denoted success and failure), and
3. The probability of a success is contant.

### Review - The geometric distribution

Suppose our experiment involves continually generating outcomes using a Bernoulli process until we see the first success.  There are two formulations for the [geometric distribution](https://en.wikipedia.org/wiki/Geometric_distribution), given below.

* Let $X$ is the number of failures observed before the first success, and
* Define $Y$ as the total number of trials until the first success.

Both random variable can be referred to as having a geometric distribution, but with different formulae (PMF, mean, var, etc.).

**We will use $X$ (number of failures) since that is what R uses.**

### Review - The Negative Binomial Distribution

The [negative binomial distribution](https://en.wikipedia.org/wiki/Negative_binomial_distribution) is a generalization of the geometric distributions, where we wait for $k$ successes before stopping.  Agian, the random variable can be either

* Let $X$ is the number of failures observed before the $k$th success, and
* Define $Y$ as the total number of trials until the $k$th success.

Both random variable can be referred to as having a negative binomial distribution, but with different formulae (PMF, mean, var, etc.).

**We will use $X$ (number of failures) since that is what R uses.**


### Summary

- The **geometric distribution** is for counting failures until *one success*.
- The **negative binomial distribution** is for counting failure until *$k$ successes.
- Convert questions about *total trials* into a question about *number of failures* (especially in R).

## Strategies for simulating waiting time problems.

1. Simulate the raw Bernoulli trials using `sample_until`, then transform into the number of success using either reshaping or `mutate` + `map`.
2. Simulate the number of successes directly using `rbinom`.

### Using `sample_until` to simulate raw outcomes

The `purrrfect` library provides the `sample_until` function, which is similar to the base `sample` function, but also includes a stopping condition.  The stopping condition can either be a literal value or a function in the `.p` argument.

**Signature:** `sample_until(x, .p, replace = FALSE, prob = NULL, .halt = 1000)`

In [1]:
?sample_until

### Example 1 - Flip a fair coin until the first head

In this case, the stopping condition can be the literal value `'H'`

In [2]:
coin <- c('H', 'T')
replicate(10, sample_until(coin, 'H', replace = TRUE))

.trial,.outcome
<dbl>,<list>
1,H
2,H
3,H
4,"T, T, H"
5,H
6,H
7,H
8,"T, H"
9,"T, T, H"


### Example 2 - Flip a fair coin until the three heads

Since we are waiting for more than one head, we need to make a helper function to determine if the sample has exactly 3 heads.  

In [ ]:
coin <- c('H', 'T')
three.heads <- \(x) num_successes(x, 'H') == 3
replicate(10, sample_until(coin, three.heads, replace = TRUE))


.trial,.outcome
<dbl>,<list>
1,"H, T, H, T, H"
2,"T, H, T, H, H"
3,"T, H, T, H, T, T, H"
4,"T, T, T, T, H, T, H, T, T, T, T, H"
5,"T, H, H, T, H"
6,"T, H, H, H"
7,"H, T, H, T, T, H"
8,"H, H, T, H"
9,"H, T, H, T, H"


### Example 3 - Roll a fair die until the first value of 4 or more.

In this case, where the set of successes is a compound event, we need to write two predicate functions
1. One that determine if a single roll is a success, and
2. A second that determines if the sample contains exactly one success.

In [4]:
is.success <- \(x) x >= 4
one.success <- \(x) num_successes(x, is.success) == 1
replicate(10, sample_until(1:6, one.success, replace = TRUE))

.trial,.outcome
<dbl>,<list>
1,"2, 5"
2,6
3,"2, 4"
4,6
5,4
6,4
7,"1, 3, 1, 1, 3, 5"
8,5
9,"3, 2, 2, 3, 5"


### <font color="red"> Exercise 2.2.1 - Rolling a 20-sided die </font>

Suppose we conduct experiments involving rolling a 20-sided die until each of the following conditions.

1. Exactly 1 roll of 20.
2. Two rolls of 20.
3. Three rolls of at least 18.

**Task.** For each stopping condition, use `replicate` and `sample_until` to generate 10 trials.

In [ ]:
# Your code here.

## Simulating the geometric and negative binomial random variables from raw trials

Recall that the random variable for both the geometric and negative binomial distributions is either

- The number of failures $X$ until the desired number of successes, or
- The total number of trials $Y$ until the desired number of successes.

Let's look at how to compute these values for a given simulating strategy.



### Converting raw outcomes

We can use `sample_until` to simulate both the geometric and negative binomial distributions by

1. Using `sample_until` on a sample space proportional to the success rate with `replace=TRUE`,
2. Converting the raw sequences into number of trials/failures, and
3. Solving the problem using the column from **2.**

Similar to the previous notebooks, we can convert to the number of trials/failures by either stacking the outcomes or using `mutate` + `map`.  We will illustrate with another free throw example.


### Example - LeBron shoots even more free throws

LeBron James is NBA player with a stored career.  Over the course of his career, he has made 73.5% of all free throw attempts.  Suppose that LeBron plans to keep attempting free throws until he makes 10 shots.  We want to model the number of missed shots before he makes his 10 shots.  We will illustrate both methods of converting the raw outcomes to number of failures.

**Parameters.**
* $p = 0.735$
* $k = 10$

#### Setting up a sample space

We need to sample from a space with a 73.5% chance of a made free throw, which will be accomplished using a probability vector

In [ ]:
p <- 0.735
k <- 10
shot <- c('S', 'F')
shot.probs <- c(p, 1 - p)
ten.made.shots <- \(x) num_successes(x, 'S') == 10
(replicate(10, sample_until(shot, ten.made.shots, replace = TRUE, prob = shot.probs))
 %>% col_num_successes(.outcome, 'S') # Verify correctness
)

.trial,.outcome,.successes
<dbl>,<list>,<int>
1,"S, S, S, S, S, F, S, S, S, F, S, S",10
2,"F, S, F, F, F, S, S, F, F, S, S, F, F, S, S, S, F, S, F, S",10
3,"F, S, S, S, S, S, S, S, S, S, S",10
4,"F, S, F, S, S, F, S, S, S, S, S, S, F, S",10
5,"S, F, S, S, S, S, S, S, S, S, S",10
6,"S, F, S, S, S, S, S, S, S, S, F, S",10
7,"S, S, S, S, S, F, F, F, S, S, S, S, S",10
8,"S, S, S, S, S, S, F, S, S, S, S",10
9,"F, S, S, F, S, F, S, F, S, S, F, S, S, F, S, F, S",10


#### Approach 1 - Simulate individual shots, reshape, and count trials

If we stack the outcomes, we can count the number of trials by grouping by the trials then using the `n()` function call in `summarise`

In [ ]:
p <- 0.735
k <- 10
shot <- c('S', 'F')
shot.probs <- c(p, 1 - p)
ten.made.shots <- \(x) num_successes(x, 'S') == 10

# Stack and count with group_by + summarise + n
N <- 10
(replicate(10, sample_until(shot, ten.made.shots, replace = TRUE, prob = shot.probs), .reshape='stack')
 %>% group_by(.trial)
 %>% summarise(total.trials = n())
 %>% mutate(num.failures = total.trials - k)
 )

.trial,total.trials,num.failures
<dbl>,<int>,<dbl>
1,16,6
2,20,10
3,13,3
4,11,1
5,11,1
6,10,0
7,13,3
8,11,1
9,16,6


#### Approach 2 - Simulate individual shots and counting trials by mapping `length`

Alternatively, we can map the `length` functions onto each outcome to get the total number of trials.

In [ ]:
p <- 0.735
k <- 10
shot <- c('S', 'F')
shot.probs <- c(p, 1 - p)
ten.made.shots <- \(x) num_successes(x, 'S') == 10

# Map length
N <- 10
(replicate(10, sample_until(shot, ten.made.shots, replace = TRUE, prob = shot.probs))
 %>% mutate(total.shots = map_int(.outcome, length),
            num.failures = total.shots - k)
 )

.trial,.outcome,total.shots,num.failures
<dbl>,<list>,<int>,<dbl>
1,"S, S, F, S, S, S, S, S, S, S, S",11,1
2,"F, S, F, S, S, S, S, S, S, F, S, S, S",13,3
3,"S, S, S, S, S, F, F, S, S, S, F, S, S",13,3
4,"S, S, S, F, S, F, S, S, F, S, S, S, S",13,3
5,"S, F, F, S, S, S, S, F, F, S, F, F, S, S, F, S, S",17,7
6,"S, S, F, S, S, S, S, S, S, F, F, F, S, F, S",15,5
7,"F, S, F, S, S, S, F, F, S, S, S, S, S, S",14,4
8,"F, S, S, F, F, S, F, F, S, F, F, F, S, S, F, S, F, S, S, S",20,10
9,"S, S, F, S, S, S, S, S, S, S, F, S",12,2


## Generating $X$ and $Y$ directly using base `R` functions

If we want to skip working with raw outcomes, we can use the `rgeom` and `rnbinom` functions to generate the number of failures directly, where we call each functions as follows

* `rgeom(n, prob)`
* `rnbinom(n, size, prob, mu)`

Here `n` is the number of experiments (use 1), `prob` is the success rate, `size` is stopping condition, i.e., number of successes.


**Important.** As noted in the help for `rgeom` and `rnbinom`, the returned value `x` is the number of failures before he first success.

In [ ]:
?rgeom

In [ ]:
?rnbinom

#### Example - Simulate the number of missed/total shots directly using `rnbinom`

Let's return to LeBron shooting free throws until 10 made shots.

In [ ]:
p <- 0.735
k <- 10
N <- 10
(replicate_int(N, rnbinom(1, k, p))
 %>% mutate(num.failures = .outcome,
            total.shots = num.failures + k)

)

.trial,.outcome,num.failures,total.shots
<dbl>,<int>,<int>,<dbl>
1,1,1,11
2,3,3,13
3,2,2,12
4,2,2,12
5,0,0,10
6,7,7,17
7,1,1,11
8,1,1,11
9,2,2,12


## Four Types of Simulation Tasks

When performing a simulation, we will generally be trying to

1. Estimate the probability of an event,
2. Estimate the cut off for some region of the distribution (i.e., inverse probability),
3. Finding the cut off for a given tail probability, or
4. Estimate all the values for the probability mass function.

In this section, we will illustrate how to peform each task after computing the number of successes.

### Estimating Probabilities

Once we have the column of containing the number of success, we can answer probability questions per usual, e.g., make a Boolean column then use `estimate_prob` or `estimate_all_prob`.

#### Example - The probability questions for LeBron

Find the probability that LeBron
1. misses exactly 5 shots before 10 made shots,
2. takes 12 total shots before 10 made shots,
3. misses between 2 and 5 shots (inclusive) before 10 made shots.


In [5]:
p <- 0.735
k <- 10
N <- 10
(replicate_int(N, rnbinom(1, k, p))
 %>% mutate(num.failures = .outcome,
            total.shots = num.failures + k)
 %>% mutate(five.misses = num.failures == 5,
            at.least.2.misses = num.failures >= 2, # Problem 2 using failures
            at.least.12.shots = total.shots >= 12, # Problem 2 using total shots
            between.2.and.5.misses = (num.failures >= 2) & (num.failures <= 5))
#  %>% estimate_all_prob # Uncomment and bump N
)

.trial,.outcome,num.failures,total.shots,five.misses,at.least.2.misses,at.least.12.shots,between.2.and.5.misses
<dbl>,<int>,<int>,<dbl>,<lgl>,<lgl>,<lgl>,<lgl>
1,3,3,13,FALSE,TRUE,TRUE,TRUE
2,2,2,12,FALSE,TRUE,TRUE,TRUE
3,4,4,14,FALSE,TRUE,TRUE,TRUE
4,3,3,13,FALSE,TRUE,TRUE,TRUE
5,5,5,15,TRUE,TRUE,TRUE,TRUE
6,3,3,13,FALSE,TRUE,TRUE,TRUE
7,3,3,13,FALSE,TRUE,TRUE,TRUE
8,4,4,14,FALSE,TRUE,TRUE,TRUE
9,11,11,21,FALSE,TRUE,TRUE,FALSE


### Estimating the Distribution

Now that we know how to simulate a binomial random variable, we can use `tabulate` to estimate the values of the probability mass function.

#### Example - Distribution of missed free throws

In [ ]:
k <- 10
p <- 0.735
N <- 100000
(replicate_int(N, rnbinom(1, k, p))
 %>% mutate(num.failures = .outcome,
            total.shots = num.failures + 1)
 %>% tabulate(num.failures)
)

X = 0,X = 1,X = 10,X = 11,X = 12,X = 13,X = 14,X = 15,X = 16,X = 19,X = 2,X = 3,X = 4,X = 5,X = 6,X = 7,X = 8,X = 9
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.04476,0.12322,0.00682,0.00348,0.00147,0.00064,0.00032,0.00015,7e-05,1e-05,0.17767,0.18764,0.16348,0.12033,0.07957,0.04904,0.02702,0.01431


### Estimating the cut off for the left tail of the distribution.

We can `summarise` the number of successes using the `quantile` function to find the approximate cut off for the left tail of the distribution.

**Example Left-Tail Task.** Find the cut off for the smallest 30% of missed shots before LeBron makes 10 free throws.

In [ ]:
k <- 10
p <- 0.735
left.tail.percent <- 0.30
N <- 100000
(replicate_int(N, rnbinom(1, k, p))
 %>% summarise(x.lower.5.percent = quantile(.outcome, left.tail.percent))
)

x.lower.5.percent
<dbl>
2


**Example Right-Tail Task.** Find the cut off for the largest 5% of missed shots before LeBron makes 10 free throws.

In [ ]:
k <- 10
p <- 0.735
left.tail.percent <- 1 - 0.05
N <- 100000
(replicate_int(N, rnbinom(1, k, p))
 %>% summarise(x.lower.5.percent = quantile(.outcome, left.tail.percent) + 1)
)

x.lower.5.percent
<dbl>
9


### Simulating the Expected Value

 As shown in previous notebooks, simulations can also be used to simulate the various expected values. Below we summarise the approach to simulating various quantities.

1. **Mean/Expected Value.** Simulate $X$ $\longrightarrow$ summarise using `mean`.
2. **Variance/Standard Deviation.** Simulate $X$ $\longrightarrow$ summarise using `var` or `sd` functions.
3. **Expected Value of a function.** Simulate $X$ $\longrightarrow$ use `mutate` to compute an `f(X)` column $\longrightarrow$ summarise using `mean`.

#### Example 1 - LeBron's mean, variance, and standard deviation number of shots

Again, we have LeBron--a 73.5% free throw shooter--shooting 10 free throws.  Let's estimate the mean, variance, and standard deviation of the number of made shots.  I will use `rbinom` to directly simulate the binomial trials.


In [ ]:
n <- 10
k <- 10
p <- 0.735
N <- 10000
(replicate_int(N, rnbinom(1, k, p))
  %>% summarise(approx.mu = mean(.outcome),
                approx.var = var(.outcome),
                approx.sd = sd(.outcome))

)

approx.mu,approx.var,approx.sd
<dbl>,<dbl>,<dbl>
3.5865,4.836401,2.199182


#### Comparing to the theortical values

Next, we can use `mutate` to compute the actual theoretical values and use `relocate` to put the respective values next to each other.

In [ ]:
k <- 10
p <- 0.735
N <- 100000
( replicate_int(N, rnbinom(1, k, p))
  %>% summarise(approx.mu = mean(.outcome),
                approx.var = var(.outcome),
                approx.sd = sd(.outcome))
  %>% mutate(actual.mu = k/p-k,
             actual.var = k*(1-p)/p^2,
             actual.sd = sqrt(actual.var))
  %>% relocate(actual.mu, .after = approx.mu)
  %>% relocate(actual.var, .after = approx.var)
  %>% relocate(actual.sd, .after = approx.sd)
)



approx.mu,actual.mu,approx.var,actual.var,approx.sd,actual.sd
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3.60445,3.605442,4.914879,4.905364,2.216953,2.214806


### Example 2 - Betting on free throws

Now suppose LeBron is betting on this scenario (shoot until 10 successes) and suppose he

- Wins \$2 for every made shot and
- Loses \$2 for every missed shot.

**Task.** Estimate LeBron's average winning in one round of the game.

In [11]:
n <- 10
k <- 10
p <- 0.735
N <- 100000
(replicate_int(N, rnbinom(1, k, p))
  %>% mutate(num.misses = .outcome,
             winnings = 2*k - 2*num.misses)
  %>% summarise(approx.mean.winnings = mean(winnings))
  %>% mutate(exact.mean.failures = k/p-k,
             exact.mean.winnings = 2*k - 2*exact.mean.failures)
  %>% select(-exact.mean.failures) # dropping a column
)

approx.mean.winnings,exact.mean.winnings
<dbl>,<dbl>
12.79844,12.78912


### <font color="red"> Exercise 2.2.2 - Waiting time tasks </font>

Historically circuit boards used in the manufacture of video game consoles are defective 5% of the time. Suppose that we plan to sample boards coming down the line until we find 3 defective bourds.  Let $Y$ be the total number of boards tested until we find three defects.

1. Determine the distribution and parameters.
2. Find $P(Y≥5)$ using each of the three approaches discussed above.
3. Find the cut off for the lowest 25% of the distribution.
4. Use  `tabulate` to estimate the PMF of $Y$.
5. Use a simulation to estimate $E(Y)$, and $SD(Y)$.



In [ ]:
# Your code here